In [1]:
#!pip install Historic-Crypto
#!pip install pandas_datareader
#!pip install yfinance

In [43]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import time
from datetime import datetime
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import math
from winsound import Beep
import yfinance as yf
from tensorflow.keras.models import load_model

# Dados
## Conseguindo os dados
### Método 1

In [12]:
# Pegando os dados do Yahoo Finance
hoje = datetime.today().strftime('%Y-%m-%d')
dados = yf.download('ETH-USD','2022-01-01', hoje)
dados

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ETH-USD: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


### Método 2

In [21]:
# Caso por algum motivo o Yahoo tiver fora do ar, dados conseguido de outro método
#dados = pd.read_csv('C:/Users/rfp20/Desktop/Tr/dados historicos.csv',sep = ',', decimal='.')

## Organizando e preparando os dados para a modelagem

In [23]:
dados = dados.dropna()
base = dados
n = base.shape[0] - 22
base, base_test= np.split(base, [n])
base_open = base.iloc[:, 1:2].values
base_high = base.iloc[:, 2:3].values
base_low = base.iloc[:, 3:4].values
base_close = base.iloc[:, 4:5].values


base_test_open = base_test.iloc[:, 1:2].values
base_test_high = base_test.iloc[:, 2:3].values
base_test_low = base_test.iloc[:, 3:4].values
base_test_close = base_test.iloc[:, 4:5].values

In [26]:
normalizador = MinMaxScaler(feature_range=(0,1))

base_open_normalizada = normalizador.fit_transform(base_open)
base_high_normalizada = normalizador.fit_transform(base_high)
base_low_normalizada = normalizador.fit_transform(base_low)
base_close_normalizada = normalizador.fit_transform(base_close)

normalizador_previsao_open = MinMaxScaler(feature_range=(0,1))
normalizador_previsao_open.fit_transform(base_open[:,0:1])

normalizador_previsao_high = MinMaxScaler(feature_range=(0,1))
normalizador_previsao_high.fit_transform(base_high[:,0:1])

normalizador_previsao_low = MinMaxScaler(feature_range=(0,1))
normalizador_previsao_low.fit_transform(base_low[:,0:1])

normalizador_previsao_close = MinMaxScaler(feature_range=(0,1))
normalizador_previsao_close.fit_transform(base_close[:,0:1])

previsores_open = []
preco_real_open = []

for i in range(90, base_open.shape[0]):
    previsores_open.append(base_open_normalizada[i-90:i, 0])
    preco_real_open.append(base_open_normalizada[i, 0])

previsores_high = []
preco_real_high = []

for i in range(90, base_high.shape[0]):
    previsores_high.append(base_high_normalizada[i-90:i, 0])
    preco_real_high.append(base_high_normalizada[i, 0])
    
previsores_low = []
preco_real_low = []

for i in range(90, base_low.shape[0]):
    previsores_low.append(base_low_normalizada[i-90:i, 0])
    preco_real_low.append(base_low_normalizada[i, 0])

previsores_close = []
preco_real_close = []

for i in range(90, base_close.shape[0]):
    previsores_close.append(base_close_normalizada[i-90:i, 0])
    preco_real_close.append(base_close_normalizada[i, 0])
    
previsores_open, preco_real_open = np.array(previsores_open), np.array(preco_real_open)
previsores_high, preco_real_high = np.array(previsores_high), np.array(preco_real_high)
previsores_low, preco_real_low = np.array(previsores_low), np.array(preco_real_low)
previsores_close, preco_real_close = np.array(previsores_close), np.array(preco_real_close)

previsores_open = np.reshape(previsores_open, (previsores_open.shape[0], previsores_open.shape[1],1))
previsores_high = np.reshape(previsores_high, (previsores_high.shape[0], previsores_high.shape[1],1))
previsores_low = np.reshape(previsores_low, (previsores_low.shape[0], previsores_low.shape[1],1))
previsores_close = np.reshape(previsores_close, (previsores_close.shape[0], previsores_close.shape[1],1))



In [27]:
base_test_open = pd.DataFrame(base_test_open)
base_test_high = pd.DataFrame(base_test_high)
base_test_low = pd.DataFrame(base_test_low)
base_test_close = pd.DataFrame(base_test_close)

base_open = pd.DataFrame(base_open)
base_high = pd.DataFrame(base_high)
base_low = pd.DataFrame(base_low)
base_close = pd.DataFrame(base_close)

In [28]:
base_completa_open = pd.concat((base_open,base_test_open), axis = 0)
base_completa_high = pd.concat((base_high,base_test_high), axis = 0)
base_completa_low = pd.concat((base_low,base_test_low), axis = 0)
base_completa_close = pd.concat((base_close,base_test_close), axis = 0)

In [29]:
entradas_open = base_completa_open[len(base_completa_open) - len(base_test_open) - 90:].values
entradas_open = entradas_open.reshape(-1,1)
entradas_open = normalizador.transform(entradas_open)

entradas_high = base_completa_high[len(base_completa_high) - len(base_test_high) - 90:].values
entradas_high = entradas_high.reshape(-1,1)
entradas_high = normalizador.transform(entradas_high)

entradas_low = base_completa_low[len(base_completa_low) - len(base_test_low) - 90:].values
entradas_low = entradas_low.reshape(-1,1)
entradas_low = normalizador.transform(entradas_low)

entradas_close = base_completa_close[len(base_completa_close) - len(base_test_close) - 90:].values
entradas_close = entradas_close.reshape(-1,1)
entradas_close = normalizador.transform(entradas_close)

X_teste_open = []
for i in range(90, 112):
    X_teste_open.append(entradas_open[i-90:i, 0])
X_teste_open = np.array(X_teste_open)
X_teste_open = np.reshape(X_teste_open, (X_teste_open.shape[0],X_teste_open.shape[1],1))

X_teste_high = []
for i in range(90, 112):
    X_teste_high.append(entradas_high[i-90:i, 0])
X_teste_high = np.array(X_teste_high)
X_teste_high = np.reshape(X_teste_high, (X_teste_high.shape[0],X_teste_high.shape[1],1))

X_teste_low = []
for i in range(90, 112):
    X_teste_low.append(entradas_low[i-90:i, 0])
X_teste_low = np.array(X_teste_low)
X_teste_low = np.reshape(X_teste_low, (X_teste_low.shape[0],X_teste_low.shape[1],1))

X_teste_close = []
for i in range(90, 112):
    X_teste_close.append(entradas_close[i-90:i, 0])
X_teste_close = np.array(X_teste_close)
X_teste_close = np.reshape(X_teste_close, (X_teste_close.shape[0],X_teste_close.shape[1],1))

# Modelagem
Foram utilizados 4 modelos para de LSTM com pequenas mudanças,cada um foi utilizado na abertura, valor máximo, valor mínimo e fechamento. Foi avaliada a precisão utilizando o RMSE e o MAE, com mais enfase nos valores do RMSE, pois muitos autores indicam esse o avaliador de precisão, entre esses autores Morettin.

Foi utilizado abertura, valor máximo, valor mínimo e fechamento, para caso se necessário a utilização de métodos como Garman Klass para a volatilidade realizada do valor previsto.

In [81]:
regressor1_open = Sequential()
regressor1_open.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_open.shape[1], 1)))
regressor1_open.add(Dropout(0.3))

regressor1_open.add(LSTM(units = 50, return_sequences = True))
regressor1_open.add(Dropout(0.3))

regressor1_open.add(LSTM(units = 50, return_sequences = True))
regressor1_open.add(Dropout(0.3))

regressor1_open.add(LSTM(units = 50))
regressor1_open.add(Dropout(0.3))
regressor1_open.add(Dense(units = 1, activation = 'sigmoid'))

regressor1_open.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])

regressor1_open.fit(previsores_open, preco_real_open, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 23s 250ms/step - loss: 0.0667 - mean_absolute_error: 0.2225
Epoch 2/200
10/10 [==============================] - 3s 267ms/step - loss: 0.0212 - mean_absolute_error: 0.1083
Epoch 3/200
10/10 [==============================] - 3s 285ms/step - loss: 0.0172 - mean_absolute_error: 0.1016
Epoch 4/200
10/10 [==============================] - 3s 271ms/step - loss: 0.0167 - mean_absolute_error: 0.1021
Epoch 5/200
10/10 [==============================] - 3s 253ms/step - loss: 0.0135 - mean_absolute_error: 0.0933
Epoch 6/200
10/10 [==============================] - 3s 276ms/step - loss: 0.0086 - mean_absolute_error: 0.0782
Epoch 7/200
10/10 [==============================] - 3s 268ms/step - loss: 0.0082 - mean_absolute_error: 0.0734
Epoch 8/200
10/10 [==============================] - 3s 261ms/step - loss: 0.0075 - mean_absolute_error: 0.0715
Epoch 9/200
10/10 [==============================] - 3s 257ms/step - loss: 0.0072 - mean_absolute_error

In [83]:
train_score = regressor1_open.evaluate(previsores_open, preco_real_open, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0009431569487787783 MSE 0.012007245793938637 RMSE


In [84]:
regressor1_high = Sequential()
regressor1_high.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_high.shape[1], 1)))
regressor1_high.add(Dropout(0.3))

regressor1_high.add(LSTM(units = 50, return_sequences = True))
regressor1_high.add(Dropout(0.3))

regressor1_high.add(LSTM(units = 50, return_sequences = True))
regressor1_high.add(Dropout(0.3))

regressor1_high.add(LSTM(units = 50))
regressor1_high.add(Dropout(0.3))
regressor1_high.add(Dense(units = 1, activation = 'sigmoid'))

regressor1_high.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])

regressor1_high.fit(previsores_high, preco_real_high, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 46s 553ms/step - loss: 0.0734 - mean_absolute_error: 0.2409
Epoch 2/200
10/10 [==============================] - 5s 524ms/step - loss: 0.0208 - mean_absolute_error: 0.1063
Epoch 3/200
10/10 [==============================] - 5s 466ms/step - loss: 0.0184 - mean_absolute_error: 0.1041
Epoch 4/200
10/10 [==============================] - 5s 494ms/step - loss: 0.0187 - mean_absolute_error: 0.1057
Epoch 5/200
10/10 [==============================] - 5s 463ms/step - loss: 0.0172 - mean_absolute_error: 0.1023
Epoch 6/200
10/10 [==============================] - 6s 569ms/step - loss: 0.0178 - mean_absolute_error: 0.1051
Epoch 7/200
10/10 [==============================] - 5s 464ms/step - loss: 0.0184 - mean_absolute_error: 0.1062
Epoch 8/200
10/10 [==============================] - 5s 478ms/step - loss: 0.0187 - mean_absolute_error: 0.1039
Epoch 9/200
10/10 [==============================] - 6s 562ms/step - loss: 0.0190 - mean_absolute_error

In [85]:
train_score = regressor1_high.evaluate(previsores_high, preco_real_high, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0021863135043531656 MSE 0.015804532915353775 RMSE


In [86]:
regressor1_low = Sequential()
regressor1_low.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_low.shape[1], 1)))
regressor1_low.add(Dropout(0.3))

regressor1_low.add(LSTM(units = 50, return_sequences = True))
regressor1_low.add(Dropout(0.3))

regressor1_low.add(LSTM(units = 50, return_sequences = True))
regressor1_low.add(Dropout(0.3))

regressor1_low.add(LSTM(units = 50))
regressor1_low.add(Dropout(0.3))
regressor1_low.add(Dense(units = 1, activation = 'sigmoid'))

regressor1_low.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])

regressor1_low.fit(previsores_low, preco_real_low, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 17s 237ms/step - loss: 0.0544 - mean_absolute_error: 0.2030
Epoch 2/200
10/10 [==============================] - 2s 228ms/step - loss: 0.0186 - mean_absolute_error: 0.1036
Epoch 3/200
10/10 [==============================] - 3s 255ms/step - loss: 0.0159 - mean_absolute_error: 0.0985
Epoch 4/200
10/10 [==============================] - 2s 239ms/step - loss: 0.0159 - mean_absolute_error: 0.1012
Epoch 5/200
10/10 [==============================] - 2s 224ms/step - loss: 0.0140 - mean_absolute_error: 0.0931
Epoch 6/200
10/10 [==============================] - 2s 245ms/step - loss: 0.0135 - mean_absolute_error: 0.0910
Epoch 7/200
10/10 [==============================] - 2s 241ms/step - loss: 0.0101 - mean_absolute_error: 0.0812
Epoch 8/200
10/10 [==============================] - 2s 229ms/step - loss: 0.0086 - mean_absolute_error: 0.0772
Epoch 9/200
10/10 [==============================] - 2s 233ms/step - loss: 0.0083 - mean_absolute_error

In [88]:
train_score = regressor1_low.evaluate(previsores_low, preco_real_low, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.001164630288258195 MSE 0.012032301165163517 RMSE


In [89]:
regressor1_close = Sequential()
regressor1_close.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_close.shape[1], 1)))
regressor1_close.add(Dropout(0.3))

regressor1_close.add(LSTM(units = 50, return_sequences = True))
regressor1_close.add(Dropout(0.3))

regressor1_close.add(LSTM(units = 50, return_sequences = True))
regressor1_close.add(Dropout(0.3))

regressor1_close.add(LSTM(units = 50))
regressor1_close.add(Dropout(0.3))
regressor1_close.add(Dense(units = 1, activation = 'sigmoid'))

regressor1_close.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])

regressor1_close.fit(previsores_close, preco_real_close, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 51s 526ms/step - loss: 0.0631 - mean_absolute_error: 0.2161
Epoch 2/200
10/10 [==============================] - 5s 499ms/step - loss: 0.0188 - mean_absolute_error: 0.1015
Epoch 3/200
10/10 [==============================] - 5s 508ms/step - loss: 0.0166 - mean_absolute_error: 0.1028
Epoch 4/200
10/10 [==============================] - 6s 639ms/step - loss: 0.0162 - mean_absolute_error: 0.0970
Epoch 5/200
10/10 [==============================] - 5s 498ms/step - loss: 0.0162 - mean_absolute_error: 0.1008
Epoch 6/200
10/10 [==============================] - 4s 444ms/step - loss: 0.0163 - mean_absolute_error: 0.0982
Epoch 7/200
10/10 [==============================] - 4s 432ms/step - loss: 0.0166 - mean_absolute_error: 0.1009
Epoch 8/200
10/10 [==============================] - 4s 358ms/step - loss: 0.0151 - mean_absolute_error: 0.0981
Epoch 9/200
10/10 [==============================] - 4s 443ms/step - loss: 0.0131 - mean_absolute_error

In [90]:
train_score = regressor1_close.evaluate(previsores_close, preco_real_close, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0020845518447458744 MSE 0.01786121167242527 RMSE


In [91]:
regressor2_open = Sequential()
regressor2_open.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_open.shape[1], 1)))
regressor2_open.add(Dropout(0.3))

regressor2_open.add(LSTM(units = 50, return_sequences = True))
regressor2_open.add(Dropout(0.3))

regressor2_open.add(LSTM(units = 50, return_sequences = True))
regressor2_open.add(Dropout(0.3))

regressor2_open.add(LSTM(units = 50))
regressor2_open.add(Dropout(0.3))

regressor2_open.add(Dense(units = 1, activation = 'sigmoid'))

regressor2_open.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor2_open.fit(previsores_open, preco_real_open, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 30s 409ms/step - loss: 0.0441 - mean_absolute_error: 0.1700
Epoch 2/200
10/10 [==============================] - 4s 403ms/step - loss: 0.0184 - mean_absolute_error: 0.1037
Epoch 3/200
10/10 [==============================] - 4s 412ms/step - loss: 0.0171 - mean_absolute_error: 0.0991
Epoch 4/200
10/10 [==============================] - 4s 369ms/step - loss: 0.0165 - mean_absolute_error: 0.0995
Epoch 5/200
10/10 [==============================] - 4s 363ms/step - loss: 0.0145 - mean_absolute_error: 0.0958
Epoch 6/200
10/10 [==============================] - 4s 401ms/step - loss: 0.0129 - mean_absolute_error: 0.0901
Epoch 7/200
10/10 [==============================] - 4s 382ms/step - loss: 0.0094 - mean_absolute_error: 0.0790
Epoch 8/200
10/10 [==============================] - 4s 370ms/step - loss: 0.0103 - mean_absolute_error: 0.0818
Epoch 9/200
10/10 [==============================] - 4s 396ms/step - loss: 0.0086 - mean_absolute_error

In [92]:
train_score = regressor2_open.evaluate(previsores_open, preco_real_open, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.001987933414056897 MSE 0.017652351409196854 RMSE


In [93]:
regressor2_high = Sequential()
regressor2_high.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_high.shape[1], 1)))
regressor2_high.add(Dropout(0.3))

regressor2_high.add(LSTM(units = 50, return_sequences = True))
regressor2_high.add(Dropout(0.3))

regressor2_high.add(LSTM(units = 50, return_sequences = True))
regressor2_high.add(Dropout(0.3))

regressor2_high.add(LSTM(units = 50))
regressor2_high.add(Dropout(0.3))

regressor2_high.add(Dense(units = 1, activation = 'sigmoid'))

regressor2_high.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor2_high.fit(previsores_high, preco_real_high, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 27s 404ms/step - loss: 0.0579 - mean_absolute_error: 0.2012
Epoch 2/200
10/10 [==============================] - 4s 377ms/step - loss: 0.0195 - mean_absolute_error: 0.1102
Epoch 3/200
10/10 [==============================] - 4s 379ms/step - loss: 0.0180 - mean_absolute_error: 0.1037
Epoch 4/200
10/10 [==============================] - 4s 373ms/step - loss: 0.0176 - mean_absolute_error: 0.1029
Epoch 5/200
10/10 [==============================] - 4s 410ms/step - loss: 0.0173 - mean_absolute_error: 0.1031
Epoch 6/200
10/10 [==============================] - 4s 390ms/step - loss: 0.0162 - mean_absolute_error: 0.1025
Epoch 7/200
10/10 [==============================] - 4s 377ms/step - loss: 0.0142 - mean_absolute_error: 0.0985
Epoch 8/200
10/10 [==============================] - 4s 376ms/step - loss: 0.0115 - mean_absolute_error: 0.0855
Epoch 9/200
10/10 [==============================] - 4s 345ms/step - loss: 0.0117 - mean_absolute_error

In [94]:
train_score = regressor2_high.evaluate(previsores_high, preco_real_high, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0029666582122445107 MSE 0.021149439737200737 RMSE


In [95]:
regressor2_low = Sequential()
regressor2_low.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_low.shape[1], 1)))
regressor2_low.add(Dropout(0.3))

regressor2_low.add(LSTM(units = 50, return_sequences = True))
regressor2_low.add(Dropout(0.3))

regressor2_low.add(LSTM(units = 50, return_sequences = True))
regressor2_low.add(Dropout(0.3))

regressor2_low.add(LSTM(units = 50))
regressor2_low.add(Dropout(0.3))

regressor2_low.add(Dense(units = 1, activation = 'sigmoid'))

regressor2_low.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor2_low.fit(previsores_low, preco_real_low, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 26s 366ms/step - loss: 0.0451 - mean_absolute_error: 0.1743
Epoch 2/200
10/10 [==============================] - 3s 343ms/step - loss: 0.0178 - mean_absolute_error: 0.1009
Epoch 3/200
10/10 [==============================] - 3s 335ms/step - loss: 0.0154 - mean_absolute_error: 0.0958
Epoch 4/200
10/10 [==============================] - 2s 230ms/step - loss: 0.0143 - mean_absolute_error: 0.0936
Epoch 5/200
10/10 [==============================] - 2s 213ms/step - loss: 0.0149 - mean_absolute_error: 0.0957
Epoch 6/200
10/10 [==============================] - 2s 220ms/step - loss: 0.0133 - mean_absolute_error: 0.0896
Epoch 7/200
10/10 [==============================] - 2s 249ms/step - loss: 0.0144 - mean_absolute_error: 0.0922
Epoch 8/200
10/10 [==============================] - 2s 221ms/step - loss: 0.0111 - mean_absolute_error: 0.0846
Epoch 9/200
10/10 [==============================] - 2s 218ms/step - loss: 0.0089 - mean_absolute_error

In [96]:
train_score = regressor2_low.evaluate(previsores_low, preco_real_low, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.003420979483053088 MSE 0.02271626517176628 RMSE


In [97]:
regressor2_close = Sequential()
regressor2_close.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_close.shape[1], 1)))
regressor2_close.add(Dropout(0.3))

regressor2_close.add(LSTM(units = 50, return_sequences = True))
regressor2_close.add(Dropout(0.3))

regressor2_close.add(LSTM(units = 50, return_sequences = True))
regressor2_close.add(Dropout(0.3))

regressor2_close.add(LSTM(units = 50))
regressor2_close.add(Dropout(0.3))

regressor2_close.add(Dense(units = 1, activation = 'sigmoid'))

regressor2_close.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor2_close.fit(previsores_close, preco_real_close, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 20s 273ms/step - loss: 0.0476 - mean_absolute_error: 0.1799
Epoch 2/200
10/10 [==============================] - 3s 296ms/step - loss: 0.0182 - mean_absolute_error: 0.1032
Epoch 3/200
10/10 [==============================] - 3s 303ms/step - loss: 0.0174 - mean_absolute_error: 0.1033
Epoch 4/200
10/10 [==============================] - 3s 282ms/step - loss: 0.0160 - mean_absolute_error: 0.0997
Epoch 5/200
10/10 [==============================] - 3s 285ms/step - loss: 0.0153 - mean_absolute_error: 0.0963
Epoch 6/200
10/10 [==============================] - 3s 315ms/step - loss: 0.0140 - mean_absolute_error: 0.0943
Epoch 7/200
10/10 [==============================] - 3s 279ms/step - loss: 0.0128 - mean_absolute_error: 0.0899
Epoch 8/200
10/10 [==============================] - 3s 295ms/step - loss: 0.0105 - mean_absolute_error: 0.0841
Epoch 9/200
10/10 [==============================] - 3s 302ms/step - loss: 0.0082 - mean_absolute_error

In [98]:
train_score = regressor2_close.evaluate(previsores_close, preco_real_close, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0022741779685020447 MSE 0.01854074001312256 RMSE


In [99]:
regressor3_open = Sequential()
regressor3_open.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_open.shape[1], 1)))
regressor3_open.add(Dropout(0.3))

regressor3_open.add(LSTM(units = 50, return_sequences = True))
regressor3_open.add(Dropout(0.3))

regressor3_open.add(LSTM(units = 50, return_sequences = True))
regressor3_open.add(Dropout(0.3))

regressor3_open.add(LSTM(units = 50))
regressor3_open.add(Dropout(0.3))

regressor3_open.add(Dense(units = 1, activation = 'linear'))

regressor3_open.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor3_open.fit(previsores_open, preco_real_open, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 17s 252ms/step - loss: 0.0204 - mean_absolute_error: 0.1075
Epoch 2/200
10/10 [==============================] - 2s 238ms/step - loss: 0.0096 - mean_absolute_error: 0.0785
Epoch 3/200
10/10 [==============================] - 2s 230ms/step - loss: 0.0076 - mean_absolute_error: 0.0692
Epoch 4/200
10/10 [==============================] - 2s 230ms/step - loss: 0.0072 - mean_absolute_error: 0.0670
Epoch 5/200
10/10 [==============================] - 3s 257ms/step - loss: 0.0061 - mean_absolute_error: 0.0627
Epoch 6/200
10/10 [==============================] - 2s 241ms/step - loss: 0.0057 - mean_absolute_error: 0.0591
Epoch 7/200
10/10 [==============================] - 2s 227ms/step - loss: 0.0062 - mean_absolute_error: 0.0625
Epoch 8/200
10/10 [==============================] - 2s 237ms/step - loss: 0.0071 - mean_absolute_error: 0.0657
Epoch 9/200
10/10 [==============================] - 2s 250ms/step - loss: 0.0066 - mean_absolute_error

In [100]:
train_score = regressor3_open.evaluate(previsores_open, preco_real_open, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0008894512429833412 MSE 0.011176984757184982 RMSE


In [101]:
regressor3_high = Sequential()
regressor3_high.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_high.shape[1], 1)))
regressor3_high.add(Dropout(0.3))

regressor3_high.add(LSTM(units = 50, return_sequences = True))
regressor3_high.add(Dropout(0.3))

regressor3_high.add(LSTM(units = 50, return_sequences = True))
regressor3_high.add(Dropout(0.3))

regressor3_high.add(LSTM(units = 50))
regressor3_high.add(Dropout(0.3))

regressor3_high.add(Dense(units = 1, activation = 'linear'))

regressor3_high.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor3_high.fit(previsores_high, preco_real_high, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 17s 238ms/step - loss: 0.0247 - mean_absolute_error: 0.1174
Epoch 2/200
10/10 [==============================] - 2s 246ms/step - loss: 0.0113 - mean_absolute_error: 0.0830
Epoch 3/200
10/10 [==============================] - 2s 235ms/step - loss: 0.0099 - mean_absolute_error: 0.0799
Epoch 4/200
10/10 [==============================] - 2s 226ms/step - loss: 0.0074 - mean_absolute_error: 0.0706
Epoch 5/200
10/10 [==============================] - 2s 226ms/step - loss: 0.0070 - mean_absolute_error: 0.0662
Epoch 6/200
10/10 [==============================] - 3s 257ms/step - loss: 0.0069 - mean_absolute_error: 0.0654
Epoch 7/200
10/10 [==============================] - 2s 228ms/step - loss: 0.0060 - mean_absolute_error: 0.0616
Epoch 8/200
10/10 [==============================] - 2s 228ms/step - loss: 0.0063 - mean_absolute_error: 0.0623
Epoch 9/200
10/10 [==============================] - 2s 225ms/step - loss: 0.0061 - mean_absolute_error

In [102]:
train_score = regressor3_high.evaluate(previsores_high, preco_real_high, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0007095163455232978 MSE 0.009464191272854805 RMSE


In [103]:
regressor3_low = Sequential()
regressor3_low.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_low.shape[1], 1)))
regressor3_low.add(Dropout(0.3))

regressor3_low.add(LSTM(units = 50, return_sequences = True))
regressor3_low.add(Dropout(0.3))

regressor3_low.add(LSTM(units = 50, return_sequences = True))
regressor3_low.add(Dropout(0.3))

regressor3_low.add(LSTM(units = 50))
regressor3_low.add(Dropout(0.3))

regressor3_low.add(Dense(units = 1, activation = 'linear'))

regressor3_low.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor3_low.fit(previsores_low, preco_real_low, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 17s 223ms/step - loss: 0.0242 - mean_absolute_error: 0.1165
Epoch 2/200
10/10 [==============================] - 2s 236ms/step - loss: 0.0077 - mean_absolute_error: 0.0699
Epoch 3/200
10/10 [==============================] - 2s 246ms/step - loss: 0.0072 - mean_absolute_error: 0.0685
Epoch 4/200
10/10 [==============================] - 2s 247ms/step - loss: 0.0062 - mean_absolute_error: 0.0616
Epoch 5/200
10/10 [==============================] - 2s 224ms/step - loss: 0.0071 - mean_absolute_error: 0.0636
Epoch 6/200
10/10 [==============================] - 2s 226ms/step - loss: 0.0064 - mean_absolute_error: 0.0618
Epoch 7/200
10/10 [==============================] - 3s 253ms/step - loss: 0.0070 - mean_absolute_error: 0.0649
Epoch 8/200
10/10 [==============================] - 2s 235ms/step - loss: 0.0070 - mean_absolute_error: 0.0651
Epoch 9/200
10/10 [==============================] - 2s 223ms/step - loss: 0.0059 - mean_absolute_error

In [104]:
train_score = regressor3_low.evaluate(previsores_low, preco_real_low, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0009308572043664753 MSE 0.01151056308299303 RMSE


In [105]:
regressor3_close = Sequential()
regressor3_close.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_close.shape[1], 1)))
regressor3_close.add(Dropout(0.3))

regressor3_close.add(LSTM(units = 50, return_sequences = True))
regressor3_close.add(Dropout(0.3))

regressor3_close.add(LSTM(units = 50, return_sequences = True))
regressor3_close.add(Dropout(0.3))

regressor3_close.add(LSTM(units = 50))
regressor3_close.add(Dropout(0.3))

regressor3_close.add(Dense(units = 1, activation = 'linear'))

regressor3_close.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor3_close.fit(previsores_close, preco_real_close, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 17s 224ms/step - loss: 0.0247 - mean_absolute_error: 0.1167
Epoch 2/200
10/10 [==============================] - 2s 222ms/step - loss: 0.0112 - mean_absolute_error: 0.0819
Epoch 3/200
10/10 [==============================] - 2s 239ms/step - loss: 0.0094 - mean_absolute_error: 0.0774
Epoch 4/200
10/10 [==============================] - 2s 235ms/step - loss: 0.0072 - mean_absolute_error: 0.0684
Epoch 5/200
10/10 [==============================] - 2s 222ms/step - loss: 0.0061 - mean_absolute_error: 0.0623
Epoch 6/200
10/10 [==============================] - 2s 224ms/step - loss: 0.0064 - mean_absolute_error: 0.0617
Epoch 7/200
10/10 [==============================] - 2s 222ms/step - loss: 0.0065 - mean_absolute_error: 0.0624
Epoch 8/200
10/10 [==============================] - 2s 248ms/step - loss: 0.0061 - mean_absolute_error: 0.0623
Epoch 9/200
10/10 [==============================] - 2s 249ms/step - loss: 0.0055 - mean_absolute_error

In [106]:
train_score = regressor3_close.evaluate(previsores_close, preco_real_close, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0009612239082343876 MSE 0.011703870259225368 RMSE


In [107]:
regressor4_open = Sequential()
regressor4_open.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_open.shape[1], 1)))
regressor4_open.add(Dropout(0.3))

regressor4_open.add(LSTM(units = 50, return_sequences = True))
regressor4_open.add(Dropout(0.3))

regressor4_open.add(LSTM(units = 50, return_sequences = True))
regressor4_open.add(Dropout(0.3))

regressor4_open.add(LSTM(units = 50))
regressor4_open.add(Dropout(0.3))

regressor4_open.add(Dense(units = 1, activation = 'linear'))

regressor4_open.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor4_open.fit(previsores_open, preco_real_open, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 21s 226ms/step - loss: 0.0175 - mean_absolute_error: 0.1047
Epoch 2/200
10/10 [==============================] - 2s 226ms/step - loss: 0.0121 - mean_absolute_error: 0.0852
Epoch 3/200
10/10 [==============================] - 2s 233ms/step - loss: 0.0082 - mean_absolute_error: 0.0707
Epoch 4/200
10/10 [==============================] - 3s 256ms/step - loss: 0.0075 - mean_absolute_error: 0.0678
Epoch 5/200
10/10 [==============================] - 2s 228ms/step - loss: 0.0095 - mean_absolute_error: 0.0773
Epoch 6/200
10/10 [==============================] - 2s 230ms/step - loss: 0.0076 - mean_absolute_error: 0.0670
Epoch 7/200
10/10 [==============================] - 2s 232ms/step - loss: 0.0072 - mean_absolute_error: 0.0681
Epoch 8/200
10/10 [==============================] - 2s 243ms/step - loss: 0.0075 - mean_absolute_error: 0.0683
Epoch 9/200
10/10 [==============================] - 2s 249ms/step - loss: 0.0062 - mean_absolute_error

In [108]:
train_score = regressor4_open.evaluate(previsores_open, preco_real_open, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0017742651980370283 MSE 0.015701057389378548 RMSE


In [109]:
regressor4_high = Sequential()
regressor4_high.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_high.shape[1], 1)))
regressor4_high.add(Dropout(0.3))

regressor4_high.add(LSTM(units = 50, return_sequences = True))
regressor4_high.add(Dropout(0.3))

regressor4_high.add(LSTM(units = 50, return_sequences = True))
regressor4_high.add(Dropout(0.3))

regressor4_high.add(LSTM(units = 50))
regressor4_high.add(Dropout(0.3))

regressor4_high.add(Dense(units = 1, activation = 'linear'))

regressor4_high.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor4_high.fit(previsores_high, preco_real_high, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 16s 233ms/step - loss: 0.0137 - mean_absolute_error: 0.0908
Epoch 2/200
10/10 [==============================] - 2s 226ms/step - loss: 0.0119 - mean_absolute_error: 0.0862
Epoch 3/200
10/10 [==============================] - 2s 228ms/step - loss: 0.0080 - mean_absolute_error: 0.0718
Epoch 4/200
10/10 [==============================] - 3s 252ms/step - loss: 0.0134 - mean_absolute_error: 0.0894
Epoch 5/200
10/10 [==============================] - 2s 229ms/step - loss: 0.0073 - mean_absolute_error: 0.0694
Epoch 6/200
10/10 [==============================] - 2s 225ms/step - loss: 0.0067 - mean_absolute_error: 0.0656
Epoch 7/200
10/10 [==============================] - 2s 223ms/step - loss: 0.0084 - mean_absolute_error: 0.0728
Epoch 8/200
10/10 [==============================] - 2s 240ms/step - loss: 0.0067 - mean_absolute_error: 0.0636
Epoch 9/200
10/10 [==============================] - 3s 248ms/step - loss: 0.0076 - mean_absolute_error

In [110]:
train_score = regressor4_high.evaluate(previsores_high, preco_real_high, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0015180142363533378 MSE 0.014461985789239407 RMSE


In [111]:
regressor4_low = Sequential()
regressor4_low.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_low.shape[1], 1)))
regressor4_low.add(Dropout(0.3))

regressor4_low.add(LSTM(units = 50, return_sequences = True))
regressor4_low.add(Dropout(0.3))

regressor4_low.add(LSTM(units = 50, return_sequences = True))
regressor4_low.add(Dropout(0.3))

regressor4_low.add(LSTM(units = 50))
regressor4_low.add(Dropout(0.3))

regressor4_low.add(Dense(units = 1, activation = 'linear'))

regressor4_low.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor4_low.fit(previsores_low, preco_real_low, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 18s 255ms/step - loss: 0.0232 - mean_absolute_error: 0.1161
Epoch 2/200
10/10 [==============================] - 3s 271ms/step - loss: 0.0090 - mean_absolute_error: 0.0763
Epoch 3/200
10/10 [==============================] - 2s 239ms/step - loss: 0.0095 - mean_absolute_error: 0.0772
Epoch 4/200
10/10 [==============================] - 2s 251ms/step - loss: 0.0088 - mean_absolute_error: 0.0739
Epoch 5/200
10/10 [==============================] - 2s 239ms/step - loss: 0.0080 - mean_absolute_error: 0.0706
Epoch 6/200
10/10 [==============================] - 2s 237ms/step - loss: 0.0084 - mean_absolute_error: 0.0712
Epoch 7/200
10/10 [==============================] - 2s 237ms/step - loss: 0.0064 - mean_absolute_error: 0.0631
Epoch 8/200
10/10 [==============================] - 2s 240ms/step - loss: 0.0077 - mean_absolute_error: 0.0679
Epoch 9/200
10/10 [==============================] - 2s 225ms/step - loss: 0.0076 - mean_absolute_error

In [112]:
train_score = regressor4_low.evaluate(previsores_low, preco_real_low, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.0016537191113457084 MSE 0.014477620832622051 RMSE


In [113]:
regressor4_close = Sequential()
regressor4_close.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores_close.shape[1], 1)))
regressor4_close.add(Dropout(0.3))

regressor4_close.add(LSTM(units = 50, return_sequences = True))
regressor4_close.add(Dropout(0.3))

regressor4_close.add(LSTM(units = 50, return_sequences = True))
regressor4_close.add(Dropout(0.3))

regressor4_close.add(LSTM(units = 50))
regressor4_close.add(Dropout(0.3))

regressor4_close.add(Dense(units = 1, activation = 'linear'))

regressor4_close.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',metrics = ['mean_absolute_error'])
regressor4_close.fit(previsores_close, preco_real_close, epochs = 200, batch_size = 32)

Epoch 1/200
10/10 [==============================] - 17s 254ms/step - loss: 0.0144 - mean_absolute_error: 0.0915
Epoch 2/200
10/10 [==============================] - 2s 240ms/step - loss: 0.0094 - mean_absolute_error: 0.0787
Epoch 3/200
10/10 [==============================] - 2s 234ms/step - loss: 0.0093 - mean_absolute_error: 0.0737
Epoch 4/200
10/10 [==============================] - 2s 226ms/step - loss: 0.0070 - mean_absolute_error: 0.0673
Epoch 5/200
10/10 [==============================] - 2s 230ms/step - loss: 0.0081 - mean_absolute_error: 0.0723
Epoch 6/200
10/10 [==============================] - 2s 251ms/step - loss: 0.0071 - mean_absolute_error: 0.0659
Epoch 7/200
10/10 [==============================] - 2s 243ms/step - loss: 0.0072 - mean_absolute_error: 0.0656
Epoch 8/200
10/10 [==============================] - 2s 227ms/step - loss: 0.0064 - mean_absolute_error: 0.0625
Epoch 9/200
10/10 [==============================] - 2s 235ms/step - loss: 0.0064 - mean_absolute_error

In [114]:
train_score = regressor4_close.evaluate(previsores_close, preco_real_close, verbose=0)
rmse = (train_score[1] ** 1/2)
print('Pontuação de Teste:', train_score[0], 'MSE', rmse,'RMSE')

Pontuação de Teste: 0.002312537981197238 MSE 0.019420621916651726 RMSE


## Salvando os modelos

In [115]:
regressor1_open.save('C:/Users/rfp20/Desktop/Tr/regressor1_open.h5')
regressor2_open.save('C:/Users/rfp20/Desktop/Tr/regressor2_open.h5')
regressor3_open.save('C:/Users/rfp20/Desktop/Tr/regressor3_open.h5')
regressor4_open.save('C:/Users/rfp20/Desktop/Tr/regressor4_open.h5')

regressor1_high.save('C:/Users/rfp20/Desktop/Tr/regressor1_high.h5')
regressor2_high.save('C:/Users/rfp20/Desktop/Tr/regressor2_high.h5')
regressor3_high.save('C:/Users/rfp20/Desktop/Tr/regressor3_high.h5')
regressor4_high.save('C:/Users/rfp20/Desktop/Tr/regressor4_high.h5')

regressor1_low.save('C:/Users/rfp20/Desktop/Tr/regressor1_low.h5')
regressor2_low.save('C:/Users/rfp20/Desktop/Tr/regressor2_low.h5')
regressor3_low.save('C:/Users/rfp20/Desktop/Tr/regressor3_low.h5')
regressor4_low.save('C:/Users/rfp20/Desktop/Tr/regressor4_low.h5')

regressor1_close.save('C:/Users/rfp20/Desktop/Tr/regressor1_close.h5')
regressor2_close.save('C:/Users/rfp20/Desktop/Tr/regressor2_close.h5')
regressor3_close.save('C:/Users/rfp20/Desktop/Tr/regressor3_close.h5')
regressor4_close.save('C:/Users/rfp20/Desktop/Tr/regressor4_close.h5')

## Carregando os modelos

In [33]:
regressor1_open = load_model('C:/Users/rfp20/Desktop/Tr/regressor1_open.h5')
regressor2_open = load_model('C:/Users/rfp20/Desktop/Tr/regressor2_open.h5')
regressor3_open = load_model('C:/Users/rfp20/Desktop/Tr/regressor3_open.h5')
regressor4_open = load_model('C:/Users/rfp20/Desktop/Tr/regressor4_open.h5')

regressor1_high = load_model('C:/Users/rfp20/Desktop/Tr/regressor1_high.h5')
regressor2_high = load_model('C:/Users/rfp20/Desktop/Tr/regressor2_high.h5')
regressor3_high = load_model('C:/Users/rfp20/Desktop/Tr/regressor3_high.h5')
regressor4_high = load_model('C:/Users/rfp20/Desktop/Tr/regressor4_high.h5')

regressor1_low = load_model('C:/Users/rfp20/Desktop/Tr/regressor1_low.h5')
regressor2_low = load_model('C:/Users/rfp20/Desktop/Tr/regressor2_low.h5')
regressor3_low = load_model('C:/Users/rfp20/Desktop/Tr/regressor3_low.h5')
regressor4_low = load_model('C:/Users/rfp20/Desktop/Tr/regressor4_low.h5')

regressor1_close = load_model('C:/Users/rfp20/Desktop/Tr/regressor1_close.h5')
regressor2_close = load_model('C:/Users/rfp20/Desktop/Tr/regressor2_close.h5')
regressor3_close = load_model('C:/Users/rfp20/Desktop/Tr/regressor3_close.h5')
regressor4_close = load_model('C:/Users/rfp20/Desktop/Tr/regressor4_close.h5')

### Inicializando algumas variáveis para a próxima parte, criação das notas que serão usadas para aviso de compra e vende posteriormente

In [116]:
data = []
preco = []

previsao1_open = []
previsao2_open = []
previsao3_open = []
previsao4_open = []
previsao_m_open = []

previsao1_high = []
previsao2_high = []
previsao3_high = []
previsao4_high = []
previsao_m_high = []

previsao1_low = []
previsao2_low = []
previsao3_low = []
previsao4_low = []
previsao_m_low = []

previsao1_close = []
previsao2_close = []
previsao3_close = []
previsao4_close = []
previsao_m_close = []

notes = {'C': 1635,
         'D': 1835,
         'E': 2060,
         'S': 1945,
         'F': 2183,
         'G': 2450,
         'A': 2750,
         'B': 3087,
         ' ': 37}

## Criando as previsões de todos os 16 modelos

In [117]:
# Tratamento para pegar o numero de linhas necessarias
entradas_open = base_completa_open.values
entradas_high = base_completa_high.values
entradas_low = base_completa_low.values
entradas_close = base_completa_close.values
    
# Formato do numpy
entradas_open = entradas_open.reshape(-1, 1)
entradas_high = entradas_high.reshape(-1, 1)
entradas_low = entradas_low.reshape(-1, 1)
entradas_close = entradas_close.reshape(-1, 1)
# Normalizando
entradas_open = normalizador.transform(entradas_open)
        entradas_high = normalizador.transform(entradas_high)
        entradas_low = normalizador.transform(entradas_low)
        entradas_close = normalizador.transform(entradas_close)
        # Criando o loop for
    X_teste_open = []
    X_teste_high = []
    X_teste_low = []
    X_teste_close = []
    # Função para criar as variaveis
    for i in range(90, entradas_open.shape[0]):
        X_teste_open.append(entradas_open[i-90:i, 0])
    # Transformando os dados   
    X_teste_open = np.array(X_teste_open)
    X_teste_open = np.reshape(X_teste_open, (X_teste_open.shape[0], X_teste_open.shape[1], 1))
    
     # Função para criar as variaveis
    for i in range(90, entradas_high.shape[0]):
        X_teste_high.append(entradas_high[i-90:i, 0])
    # Transformando os dados   
    X_teste_high = np.array(X_teste_high)
    X_teste_high = np.reshape(X_teste_high, (X_teste_high.shape[0], X_teste_high.shape[1], 1))
    
     # Função para criar as variaveis
    for i in range(90, entradas_low.shape[0]):
        X_teste_low.append(entradas_low[i-90:i, 0])
    # Transformando os dados   
    X_teste_low = np.array(X_teste_low)
    X_teste_low = np.reshape(X_teste_low, (X_teste_low.shape[0], X_teste_low.shape[1], 1))
    
     # Função para criar as variaveis
    for i in range(90, entradas_close.shape[0]):
        X_teste_close.append(entradas_close[i-90:i, 0])
    # Transformando os dados   
    X_teste_close = np.array(X_teste_close)
    X_teste_close = np.reshape(X_teste_close, (X_teste_close.shape[0], X_teste_close.shape[1], 1))
    
    
    # Realizando a previsão
    
    previsao1_open = regressor1_open.predict(X_teste_open)
    previsao2_open = regressor2_open.predict(X_teste_open)
    previsao3_open = regressor3_open.predict(X_teste_open)
    previsao4_open = regressor4_open.predict(X_teste_open)
    
    previsao1_high = regressor1_high.predict(X_teste_high)
    previsao2_high = regressor2_high.predict(X_teste_high)
    previsao3_high = regressor3_high.predict(X_teste_high)
    previsao4_high = regressor4_high.predict(X_teste_high)
    
    previsao1_low = regressor1_low.predict(X_teste_low)
    previsao2_low = regressor2_low.predict(X_teste_low)
    previsao3_low = regressor3_low.predict(X_teste_low)
    previsao4_low = regressor4_low.predict(X_teste_low)
    
    previsao1_close = regressor1_close.predict(X_teste_close)
    previsao2_close = regressor2_close.predict(X_teste_close)
    previsao3_close = regressor3_close.predict(X_teste_close)
    previsao4_close = regressor4_close.predict(X_teste_close)
    
    # Desnormalizando os dados
    previsao1_open = normalizador.inverse_transform(previsao1_open)
    previsao2_open = normalizador.inverse_transform(previsao2_open)
    previsao3_open = normalizador.inverse_transform(previsao3_open)
    previsao4_open = normalizador.inverse_transform(previsao4_open)
       
    previsao1_high = normalizador.inverse_transform(previsao1_high)
    previsao2_high = normalizador.inverse_transform(previsao2_high)
    previsao3_high = normalizador.inverse_transform(previsao3_high)
    previsao4_high = normalizador.inverse_transform(previsao4_high)
    
    previsao1_low = normalizador.inverse_transform(previsao1_low)
    previsao2_low = normalizador.inverse_transform(previsao2_low)
    previsao3_low = normalizador.inverse_transform(previsao3_low)
    previsao4_low = normalizador.inverse_transform(previsao4_low)
    
    previsao1_close = normalizador.inverse_transform(previsao1_close)
    previsao2_close = normalizador.inverse_transform(previsao2_close)
    previsao3_close = normalizador.inverse_transform(previsao3_close)
    previsao4_close = normalizador.inverse_transform(previsao4_close)
    
    # Ultimo Registro
    
    previsao1_open = float(previsao1_open[len(previsao1_open)-1])
    previsao2_open = float(previsao2_open[len(previsao2_open)-1])
    previsao3_open = float(previsao3_open[len(previsao3_open)-1])
    previsao4_open = float(previsao4_open[len(previsao4_open)-1])
    
    previsao1_high = float(previsao1_high[len(previsao1_high)-1])
    previsao2_high = float(previsao2_high[len(previsao2_high)-1])
    previsao3_high = float(previsao3_high[len(previsao3_high)-1])
    previsao4_high = float(previsao4_high[len(previsao4_high)-1])
    
    previsao1_low = float(previsao1_low[len(previsao1_low)-1])
    previsao2_low = float(previsao2_low[len(previsao2_low)-1])
    previsao3_low = float(previsao3_low[len(previsao3_low)-1])
    previsao4_low = float(previsao4_low[len(previsao4_low)-1])
    
    previsao1_close = float(previsao1_close[len(previsao1_close)-1])
    previsao2_close = float(previsao2_close[len(previsao2_close)-1])
    previsao3_close = float(previsao3_close[len(previsao3_close)-1])
    previsao4_close = float(previsao4_close[len(previsao4_close)-1])
    
    
    media_open = (previsao1_open+previsao2_open+previsao3_open+previsao4_open)/4
    
    media_high = (previsao1_high+previsao2_high+previsao3_high+previsao4_high)/4

    media_low = (previsao1_low+previsao2_low+previsao3_low+previsao4_low)/4

    media_close = (previsao1_close+previsao2_close+previsao3_close+previsao4_close)/4
    
    print("Previsão Open P1: ", previsao1_open, "P2: ",previsao2_open, "P3: ", previsao3_open, "P4: ", previsao4_open,"Média: ",media_open)
    print("Previsão High P1: ", previsao1_high, "P2: ",previsao2_high, "P3: ", previsao3_high, "P4: ", previsao4_high,"Média: ",media_high)
    print("Previsão Open P1: ", previsao1_low, "P2: ",previsao2_low, "P3: ", previsao3_low, "P4: ", previsao4_low,"Média: ",media_low)
    print("Previsão Close P1: ", previsao1_close, "P2: ",previsao2_close, "P3: ", previsao3_close, "P4: ", previsao4_close, "Média: ",media_close)  
    
    break

10/10 [==============================] - 5s 103ms/step
Previsão Open P1:  1360.2847900390625 P2:  1270.3170166015625 P3:  1597.039794921875 P4:  1467.3336181640625 Média:  1423.7438049316406
Previsão High P1:  1636.8016357421875 P2:  1186.1224365234375 P3:  1735.9796142578125 P4:  1611.1068115234375 Média:  1542.5026245117188
Previsão Open P1:  1491.721435546875 P2:  1455.259521484375 P3:  1584.721923828125 P4:  1496.2869873046875 Média:  1506.9974670410156
Previsão Close P1:  1417.056884765625 P2:  1461.05712890625 P3:  1660.5672607421875 P4:  1497.1396484375 Média:  1508.9552307128906


## Variáveis para o alerta
preco_c é para colocar o valor mínimo previsto
preco_v é colocar o valor máximo previsto

o api_key e api_secret é para colocar os dados do API da Binance


In [122]:
preco_c = 1600
preco_v = 1735
api_key = "colocar aqui a chave"
api_secret = "colocar aqui a chave"

client = Client(api_key, api_secret)

### Metodo de alertar quando valor de compra ou venda ultrapassar o valor esperado
Infelizmente meu API não aceita compra e venda automaticamente, então tem que se colocar manualmente as compras e vendas no site da Binance

In [ ]:
while(True):
    df=pd.DataFrame(client.get_all_tickers())
    df=df.set_index("symbol")
    df["price"]=df["price"].astype("float32")
    df.index=df.index.astype("string")
    preco = df.loc["ETHUSDT"]
    preco = preco.loc["price"]
    
    if preco <= preco_c:
        print("\n")
        print("Comprou!")
        print("\n")
        melodie = 'BBBB SSSS EEEE'
        for note in melodie:
            Beep(notes[note], 20)
            
    
    if preco >= preco_v:
        print("\n")
        print("Vendeu!")
        print("\n")
        melodie = 'AAAA FFFFF GGGG'
        for note in melodie:
            Beep(notes[note], 20)
time.sleep(600)
            